Calculate DJF stationary wave amplitude for four reanalysis datasets: ERA5, ERAInterim, MERRA2 and JRA55.  Amplitude is the root mean squared meriodional wind between 200-300E, 25N-37N after filtering to retain only zonal scales greater than 3 (zonal wavenumbers 4 to 32 in practise)

(This works on a local archive in the Climate and Global Dynamics Laboratory at NCAR.  File paths would have to be changed to run this somewhere else)

In [16]:
import importlib
import xarray as xr
import numpy as np
import xesmf as xe
import sys
import warnings

from ecpaper_utils import calendar_utils as cal
from ecpaper_utils import readdata_utils as read
from ecpaper_utils import filter_utils as filt
from ecpaper_utils import averaging_utils as avg

importlib.reload(read)
importlib.reload(cal)
importlib.reload(filt)
importlib.reload(avg)

warnings.filterwarnings('ignore')

outpath="/project/cas/islas/python/ecpaper2020/DATASORT/VWIND/DATA/"

In [17]:
plevuse=300 # in hPa
ybegp=1979 ; monbegp=1 ; yendp=2014 ; monendp=12
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
nmonths = (yendp-ybegp-1)*12 + (12-monbegp+1) + monendp

# 1 deg grid
grid_out = xr.Dataset({'lat': (['lat'], np.arange(-90,91,1.0)), 'lon': (['lon'], np.arange(0,360,1.0))})
nlon = grid_out["lon"].size ; nlat=grid_out["lat"].size

wgtfile = "/project/cas/islas/python/ecpaper2020/DATASORT/VWIND/tmp/wgtfile_obs.nc"

ERA5

In [ ]:
filepath="/project/haggis/ERA5/mon/V/*.nc"
v=read.read_1lev(filepath, datebegp, dateendp, plevuse)
if (v.time.size != nmonths):
    print("expected "+str(nmonths)+" months, got"+str(v.time.size))
    sys.exit()
vdjfts=cal.season_ts(v, "va", season="DJF")
vdjf=cal.season_mean(v, "va", season="DJF")
regridder = xe.Regridder(vdjf, grid_out, 'bilinear', periodic=True, reuse_weights=False, filename=wgtfile)
vdjfinterp = regridder(vdjf)
vdjftsinterp = regridder(vdjfts)
vk4to32 = filt.filterk(vdjfinterp, 4, 32)
vamp_era5 = np.sqrt(avg.cosweightlonlat(vk4to32**2, 200., 300., 20., 40.))
vamp_era5=xr.DataArray(vamp_era5, name='vamp_era5')
vamp_era5.to_netcdf(outpath+"/vampobs.nc")
vdjfts = xr.DataArray(vdjftsinterp, name='vdjfts')
vdjfts.to_netcdf(outpath+"/v300timeseries_ERA5.nc")

ERAInterim

In [ ]:
filepath="/project/haggis/ERAInterim/analysis/plev/v/mon/*.nc"
v=read.read_1lev(filepath, datebegp, dateendp, plevuse)
if (v.time.size != nmonths):
    print("expected "+str(nmonths)+" months, got"+str(v.time.size))
    sys.exit()
vdjfts=cal.season_ts(v, "va", season="DJF")
vdjf=cal.season_mean(v, "va", season="DJF")
regridder = xe.Regridder(vdjf, grid_out, 'bilinear', periodic=True, reuse_weights=False, filename=wgtfile)
vdjfinterp = regridder(vdjf)
vdjftsinterp = regridder(vdjfts)
vk4to32 = filt.filterk(vdjfinterp, 4, 32)
vamp_erai = np.sqrt(avg.cosweightlonlat(vk4to32**2, 200., 300., 20., 40.))
vamp_erai=xr.DataArray(vamp_erai, name='vamp_erai')
vamp_erai.to_netcdf(outpath+"/vampobs.nc", mode="a")
vdjfts = xr.DataArray(vdjftsinterp, name='vdjfts')
vdjfts.to_netcdf(outpath+"/v300timeseries_ERAI.nc")

JRA55

In [13]:
filepath="/project/haggis/JRA-55/mon/va/*.nc"
v=read.read_1lev(filepath, datebegp, dateendp, plevuse)
if (v.time.size != nmonths):
    print("expected "+str(nmonths)+" months, got"+str(v.time.size))
    sys.exit()
vdjfts=cal.season_ts(v, "va", season="DJF")
vdjf=cal.season_mean(v, "va", season="DJF")
regridder = xe.Regridder(vdjf, grid_out, 'bilinear', periodic=True, reuse_weights=False, filename=wgtfile)
vdjfinterp = regridder(vdjf)
vdjftsinterp = regridder(vdjfts)
vk4to32 = filt.filterk(vdjfinterp, 4, 32)
vamp_jra = np.sqrt(avg.cosweightlonlat(vk4to32**2, 200., 300., 20., 40.))
vamp_jra=xr.DataArray(vamp_jra, name='vamp_jra')
vamp_jra.to_netcdf(outpath+"/vampobs.nc", mode="a")
vdjfts = xr.DataArray(vdjftsinterp, name='vdjfts')
vdjfts.to_netcdf(outpath+"/v300timeseries_JRA.nc")

MERRA2

In [14]:
ybegp=1980 ; monbegp=1 ; yendp=2014 ; monendp=12
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
nmonths=(yendp-ybegp-1)*12 + (12-monbegp+1)+monendp

In [15]:
filepath="/project/haggis/MERRA2/plev/mon/yearly/V/ana/*.nc"
v=read.read_1lev(filepath, datebegp, dateendp, plevuse)
if (v.time.size != nmonths):
    print("expected "+str(nmonths)+" months, got"+str(v.time.size))
    sys.exit()
vdjfts = cal.season_ts(v, "va", season="DJF")
vdjf=cal.season_mean(v, "va", season="DJF")
regridder = xe.Regridder(vdjf, grid_out, 'bilinear', periodic=True, reuse_weights=False, filename=wgtfile)
vdjfinterp = regridder(vdjf)
vdjftsinterp = regridder(vdjfts)
vk4to32 = filt.filterk(vdjfinterp, 4, 32)
vamp_merra = np.sqrt(avg.cosweightlonlat(vk4to32**2, 200., 300., 20., 40.))
vamp_merra=xr.DataArray(vamp_merra, name='vamp_merra')
vamp_merra.to_netcdf(outpath+"/vampobs.nc", mode="a")
vdjfts = xr.DataArray(vdjftsinterp, name='vdjfts')
vdjfts.to_netcdf(outpath+"/v300timeseries_MERRA2.nc")